# Lec 23-25: Meta and Aspect Oriented Programming

## Agenda

* Meta Programming
* Decorators
* Aspect-oriented Programming

We have seen how one can *declare* classes with the leading "class" keyword to support object-oriented programming. 

We have also seen that "everything in python is an object", and we can visualize all objects in Python with three mini-worlds: the Meta Class world, where "type" lives; the class world, where regular classes live; and the instance world, where instances of a class live. They objects can be created when Python compiler parse the code you write, as long as you follow the Python syntax, just like other programming languages. 

Today we are going to introduct some *super power* so to speak, which gives programmer similar power of a compiler, which *dynamically* create language constructs, as if you wrote them. This capability is called "meta-programming", and modern programming language has such capabilities with various degrees. 

One example of meta programming that you all have *USED* without realizing is the C Macros. With macros, you can actually define complex functions, and when applied, these functions will be expanded, sometimes recursively, into program text. This expansion is done by a tool called the C Preprocessor, but to the C compiler, the expanded text is as if you wrote it yourself. 

The macro approach however, is a "poorman"'s approach, as it is error prone and has to go through another tool. The native meta-programming capability that comes with the language itself is better. We will explore what is available in Python and hopefully increase your awareness in this aspect of langauge expressive power. 


## Class Factory

We will first look at how to create a class *without* actually writing them.
  
### Old fashioned way: Declaration

In [ ]:
class Foo() : 
    def say_foo( self ) :
        print( 'foo' )

Foo().say_foo()
hasattr( Foo, 'say_foo' )

## Using a Function

In [ ]:
def class_with_method( func ) :
   class klass: pass    # this is just an empty class 
      
   setattr( klass, func.__name__, func )    # this is a back door!
   return klass                             # klass is an class, and class is object

def say_foo(self): 
   print( 'foo' )

Foo = class_with_method( say_foo )
Foo().say_foo()
hasattr( Foo, 'say_foo' )

So what has happened in the stranage code above? 

We see that Foo just behave like a regular class in that:

1. You can create an instance "foo = Foo()"
2. You can call a method on the instance "foo.say_foo()"

It seems that we  have written:

~~~
class Foo() :
    def say_foo( self ) :
        print( 'foo' )
~~~

Except we didn't!

So who did this? It is the function "class_with_method". We just
assembled a class dynamically by calling "class_with_method". But 
this time it is not the compiler/parser, but YOU, the programmer.
Are you convinced that OO is nothing but a glorified dictionary with predefined syntactial sugar/convention? 

### Another way: Using 'type' meta class.

Recall one of the *root object*, 'type'? It is called a *meta class*, because it is the default type of all other classes, which in turn are types of their instances?

Since a class is an instance of 'type', we can create class by instantiation!
And 'type' is equipped with a constructor, so during instantiation, you can pass parameters to the constructor. 


In [ ]:
X = type( 'X', (), { 'foo' : lambda self:'foo' } )
X

In [ ]:
X().foo()
hasattr( X, 'foo' )

Look at what we passed into the constructor

1. The name 'X': This is as if you say "class X";
2. A tuple, although it is empty here, for the set of base classes of X;
3. A dictionary of key value pairs, in this case we defined the method 'foo';

It is interesting to know the value for 'foo' is supposed to be a function, here we pass an *annonymous function* using the lambda expression, which we will dive into more detail when we talk about functional programming. For now just remember that it is an *expression* with a function value. This is equivalent to: 

~~~
def foo( self ) : 
    return 'foo'
foo
~~~

With one instantiation of the meta class 'type', we manage to create
a class (since it is an instance of a meta class), without actually write *textually* the following: 

~~~
class foo() :
    def foo( self ) : 
        return 'foo'
~~~

Or maybe this is exactly what Python does as it is parsing your textual code above?!

### Another Way: Define your Own Meta Class

One can personalize the behavior of a 'type' simply by *inheriting* from 'type', just the *the same way* as you would personalize the behavior of a regular class by inheriting from it. 

In [ ]:
class ChattyType( type ) :
    def __new__(cls, name, bases, dct):
        print( "Allocating memory for class", name )
        return type.__new__( cls, name, bases, dct )
    
    def __init__(cls, name, bases, dct):
        print( "Init'ing (configuring) class", name )
        super( ChattyType, cls ).__init__(name, bases, dct)

X = ChattyType( 'X',(), {'foo':lambda self:'foo'} )

In [ ]:
X, X().foo()

The typical behavior of a meta class includes how to "new" an object, that is, how to allocate memory for it; and  how to "construct" an object, that is, how to initialize the object content after its memory is allocated and available. We usually do not worry about it, since the system built-in 'type' 
has already taken care of it. 

But what if you need to change this behavior. See the code above. 
But when will you ever be wanting to do this? Well, you will know when you actually need it! I am just planting a seed in your head.

But with your own meta class, you can manufacture a class, the same way as we use the 'type' meta class.

Note that for the methods we add to the meta classes, the first parameter is "cls", which takes the usual place of "self", since it represents an instance of the meta class, which is a class. The method itself is nothing but a function, but since it is applied on a class, this is equivlanet to the C++ static method (those without object instance)! 

We can see from the next example as well. Here we can call "whoami" directly on the 'Foo' class, not on its instance. Calling it on 'Printable' will not make sense.

In [ ]:
class Printable( type ):
    def whoami( cls ) : 
        print( "I am a", cls.__name__ )
    
Foo = Printable( 'Foo', (), {} )
Foo.whoami()

In [ ]:
 Printable.whoami()

### More Meta-fu

We can *declare* the meta class of a regular class, by passing a keyword parameter as shoen in the following example. 

In [ ]:
class Bar( object, metaclass=Printable ):
    def foo_method(self): print( 'foo' )

In [ ]:
Bar.whoami()

In [ ]:
Bar().foo_method()

Note that difference of the two method calls. One is on the class object, the other is on an (regular) instance object!

## Decorator

* You have seen it!

~~~
@route('/:name')
def index(name='World'):
    return '<b>Hello %s!</b>' % name
~~~

* Ever wonder what they are?
  - modify the function that is defined immediately after
  
* Disclaimer: you can do it without them -- syntactic sugar
  - but life is a lot easier with them

In [ ]:
class C:
    def foo(cls, y):
        print( "classmethod", cls, y )
    foo = classmethod(foo)

C.foo('hey')

Note what has happend in the above code:

1. The original foo() is an instance method within the class C scope.
2. We called a builtin function "classmethod", which takes a function as an argument, and a function.
3. That returned function replace the original 'foo' attribute.
4. Now you can call foo() on the class C, not an instance of C.

The builtin classmethod() function seems like a "operator", or function on function that returns a function!

Let's see if we can build our own.

In [ ]:
def enhanced(method):
    def new(self, y):
        print( "I am enhanced" )
        return method(self, y)
    return new
    
class C:
    def bar(self, x):
        print( "some method says:", x )
    bar = enhanced(bar)

C().bar( 'hey' )

* Recurring pattern: Let's invent some syntactical sugar:
     - avoid repeating the method name,
     - put it near the first mention of the method

In [ ]:
class C:
    @classmethod
    def foo(cls, y):
        print( "classmethod", cls, y )
        
    @enhanced
    def bar(self, x):
        print( "some method says:", x )


This is called a *decorator*! Essentially better syntax for what we have done earlier. But now you, as a programmer, is *extending* the language as if you are introducing a new *keyword* with user-defined behavior.

* Work for regular functions too

* Can be chained

~~~
@synchronized
@logging
def myfunc(arg1, arg2, ...):
      # ...do something
      # decorators are equivalent to ending with:
      #    myfunc = synchronized(logging(myfunc))
      # Nested in that declaration order
~~~

### Misuse of Decorators

* Not returning a function



In [ ]:
def spamdef(fn):
    print( "spam, spam, spam" )

@spamdef
def useful(a, b):
    print( a**2 + b**2 )


In [ ]:
useful(3, 4)

* Not returning a *meaningful* function


In [ ]:
def spamrun(fn):
    def sayspam(*args):
        print( "spam, spam, spam" )
    return sayspam

@spamrun
def useful(a, b):
    print( a**2 + b**2 )

useful( 3, 4 )

* Correct one

In [ ]:
def addspam(fn):
    def new(*args):
        print( "spam, spam, spam" )
        return fn( *args )
    return new

@addspam
def useful(a, b):
    print( a**2 + b**2 )

useful( 3, 4 )

### Using Decorators

In [ ]:
def elementwise(fn):
    def newfn(arg):
        if hasattr(arg,'__getitem__'):  # is a Sequence
            return type(arg)(map(fn, arg))
        else:
            return fn(arg)
    return newfn

@elementwise
def compute(x):
    return x**3 - 1

In [ ]:
compute( 5 )

In [ ]:
compute([1,2,3])

In [ ]:
compute((1,2,3))

## Aspect Oriented Programming

 * Seperatating cross-cutting concerns
   - Common across a number of classes, methods and functions

 * Category of aspects
   - Debugging: logging function argments, entry and exit
   - Type safty checks
   - Deprecation warnings
   - Database transactions
   - Authorization
   - Profiling

In [ ]:
def trace( aFunc ):
    """Trace entry, exit and exceptions."""
    def loggedFunc( *args, **kw ):
        print( "enter", aFunc.__name__ )
        try:
            result= aFunc( *args, **kw )
        except Exception as e:
            print( "exception", aFunc.__name__, e )
            raise
        print( "exit" ), aFunc.__name__
        return result
    loggedFunc.__name__= aFunc.__name__
    loggedFunc.__doc__= aFunc.__doc__
    return loggedFunc

In [ ]:
class MyClass( object ):
    @trace
    def __init__( self, someValue ):
        """Create a MyClass instance."""
        self.value= someValue
    @trace
    def doSomething( self, anotherValue ):
        """Update a value."""
        self.value += anotherValue
        return self.value

a = MyClass( 0 )
a.doSomething(1)
a.doSomething(2)
a.doSomething(10)

### Decorator Tool

In [ ]:
from decorator import decorator

@decorator
def addspam(f, *args, **kws):
    print( "spam, spam, spam" )
    return f(*args, **kws)

@addspam
def useful(a, b): return( a**2 + b**2 )

In [ ]:
useful.__name__

In [ ]:
useful( 2, 3 )

### Usage in Bottle

~~~
from bottle import route, run

@route('/:name')
def index(name='World'):
    return '<b>Hello %s!</b>' % name

run(host='localhost', port=8080)
~~~

## Recap

 * Everything is an object, but objects are different
   - Meta classes
   - classes
   - non-type instances
   - relations
       - instanceof
       - typeof

 * Meta Programming
   - Build your own "class factory" with meta classes
   - Build your own "keyword" with decorators
   - Aspect-oriented programming by identifying & defining cross-cutting behavior 
